<a href="https://colab.research.google.com/github/LaJeremi/Tensorflow-nlp-tutorial-Practice-/blob/main/09.%20Word%20Embedding/09_14_%EC%8B%A4%EC%A0%84!_%ED%95%9C%EA%B5%AD%EC%96%B4_%EC%9C%84%ED%82%A4%ED%94%BC%EB%94%94%EC%95%84%EB%A1%9C_Word2Vec_%ED%95%99%EC%8A%B5%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 09-14 실전! 한국어 위키피디아로 Word2Vec 학습하기

출처: https://wikidocs.net/152606

# **1. 위키피디아로부터 데이터 다운로드 및 통합**

위키피디아로부터 데이터를 파싱하기 위한 파이썬 패키지인 wikiextractor를 설치합니다.


In [ ]:
pip install wikiextractor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 KB 1.6 MB/s eta 0:00:00


위키피디아 데이터를 다운로드 한 후에 전처리에서 사용할 형태소 분석기인 Mecab을 설치합니다.

In [ ]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 3.47 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 81.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 45.6 MB/s eta 0:00:00
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2023-02-15 06:34:59--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22cd:e0db, 2406:da00:ff00::

위키피디아 덤프(위키피디아 데이터)를 다운로드합니다.



In [ ]:
!wget https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles.xml.bz2

--2023-02-15 06:37:22--  https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.142, 2620:0:861:2:208:80:154:142
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 884286084 (843M) [application/octet-stream]
Saving to: ‘kowiki-latest-pages-articles.xml.bz2’

kowiki-latest-pages 100%[===================>] 843.32M  4.72MB/s    in 3m 20s  

2023-02-15 06:40:43 (4.22 MB/s) - ‘kowiki-latest-pages-articles.xml.bz2’ saved [884286084/884286084]



위키익스트랙터를 사용하여 위키피디아 덤프를 파싱합니다.



In [ ]:
!python -m wikiextractor.WikiExtractor kowiki-latest-pages-articles.xml.bz2

INFO: Preprocessing 'kowiki-latest-pages-articles.xml.bz2' to collect template definitions: this may take some time.
INFO: Preprocessed 100000 pages
INFO: Preprocessed 200000 pages
INFO: Preprocessed 300000 pages
INFO: Preprocessed 400000 pages
INFO: Preprocessed 500000 pages
INFO: Preprocessed 600000 pages
INFO: Preprocessed 700000 pages
INFO: Preprocessed 800000 pages
INFO: Preprocessed 900000 pages
INFO: Preprocessed 1000000 pages
INFO: Preprocessed 1100000 pages
INFO: Preprocessed 1200000 pages
INFO: Preprocessed 1300000 pages
INFO: Preprocessed 1400000 pages
INFO: Preprocessed 1500000 pages
INFO: Preprocessed 1600000 pages
INFO: Preprocessed 1700000 pages
INFO: Preprocessed 1800000 pages
INFO: Loaded 61795 templates in 251.9s
INFO: Starting page extraction from kowiki-latest-pages-articles.xml.bz2.
INFO: Using 11 extract processes.
INFO: Extracted 100000 articles (2211.0 art/s)
INFO: Extracted 200000 articles (3257.3 art/s)
INFO: Extracted 300000 articles (3614.5 art/s)
INFO: Extr

현재 경로에 있는 디렉토리와 파일들의 리스트를 받아옵니다.



In [ ]:
%ls

images/                                    LICENSE
install_mecab-ko_on_colab190912.sh         README.md
install_mecab-ko_on_colab_light_220429.sh  text/
kowiki-latest-pages-articles.xml.bz2


In [ ]:
%ls text/AA


wiki_00  wiki_12  wiki_24  wiki_36  wiki_48  wiki_60  wiki_72  wiki_84  wiki_96
wiki_01  wiki_13  wiki_25  wiki_37  wiki_49  wiki_61  wiki_73  wiki_85  wiki_97
wiki_02  wiki_14  wiki_26  wiki_38  wiki_50  wiki_62  wiki_74  wiki_86  wiki_98
wiki_03  wiki_15  wiki_27  wiki_39  wiki_51  wiki_63  wiki_75  wiki_87  wiki_99
wiki_04  wiki_16  wiki_28  wiki_40  wiki_52  wiki_64  wiki_76  wiki_88
wiki_05  wiki_17  wiki_29  wiki_41  wiki_53  wiki_65  wiki_77  wiki_89
wiki_06  wiki_18  wiki_30  wiki_42  wiki_54  wiki_66  wiki_78  wiki_90
wiki_07  wiki_19  wiki_31  wiki_43  wiki_55  wiki_67  wiki_79  wiki_91
wiki_08  wiki_20  wiki_32  wiki_44  wiki_56  wiki_68  wiki_80  wiki_92
wiki_09  wiki_21  wiki_33  wiki_45  wiki_57  wiki_69  wiki_81  wiki_93
wiki_10  wiki_22  wiki_34  wiki_46  wiki_58  wiki_70  wiki_82  wiki_94
wiki_11  wiki_23  wiki_35  wiki_47  wiki_59  wiki_71  wiki_83  wiki_95


text라는 디렉토리 안에는 또 어떤 디렉토리들이 있는지 파이썬을 사용하여 확인해봅시다.

In [ ]:
import os
import re

In [ ]:
os.listdir('text')


['AB', 'AA', 'AG', 'AJ', 'AC', 'AI', 'AD', 'AE', 'AH', 'AF']

In [ ]:
%ls text/AA
# 디렉토리의 파일들을 확인한다. 실제 구동시 아래와 다름. 원본 사이트 참조할 것. 

['AB', 'AA', 'AG', 'AJ', 'AC', 'AI', 'AD', 'AE', 'AH', 'AF']

AA라는 디렉토리의 파일들을 확인해봅시다.



텍스트 파일로 변환된 위키피디아 한국어 덤프는 총 6개의 디렉토리로 구성되어져 있습니다. AA ~ AF의 디렉토리로 각 디렉토리 내에는 'wiki_00 ~ wiki_약 90내외의 숫자'의 파일들이 들어있습니다. 다시 말해 각 디렉토리에는 약 90여개의 파일들이 들어있습니다. 각 파일들을 열어보면 이와 같은 구성이 반복되고 있습니다.

* 예를 들어서 AA 디렉토리의 wiki_00 파일을 읽어보면, 지미 카터에 대한 내용이 나옵니다.

<doc id="5" url="https://ko.wikipedia.org/wiki?curid=5" title="지미 카터">
지미 카터
제임스 얼 "지미" 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령(1977년 ~ 1981년)이다.
지미 카터는 조지아 주 섬터 카운티 플레인스 마을에서 태어났다. 조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대
위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다.
... 이하 중략...
</doc>

-----

이제 이 6개 AA ~ AF 디렉토리 안의 wiki 숫자 형태의 수많은 파일들을 하나로 통합하는 과정을 진행해야 합니다. AA ~ AF 디렉토리 안의 모든 파일들의 경로를 파이썬의 리스트 형태로 저장합니다.

In [ ]:
def list_wiki(dirname):
    filepaths = []
    filenames = os.listdir(dirname)
    for filename in filenames:
        filepath = os.path.join(dirname, filename)

        if os.path.isdir(filepath):
            # 재귀 함수
            filepaths.extend(list_wiki(filepath))
        else:
            find = re.findall(r"wiki_[0-9][0-9]", filepath)
            if 0 < len(find):
                filepaths.append(filepath)
    return sorted(filepaths)

In [ ]:
filepaths = list_wiki('text')


총 파일의 개수를 확인해봅시다.



In [ ]:
len(filepaths)


941

. 이제 output_file.txt라는 파일에 951개의 파일을 전부 하나로 합칩니다.

In [ ]:
with open("output_file.txt", "w") as outfile:
    for filename in filepaths:
        with open(filename) as infile:
            contents = infile.read()
            outfile.write(contents)

파일을 읽고 10줄만 출력해보겠습니다.



In [ ]:
f = open('output_file.txt', encoding="utf8")

i = 0
while True:
    line = f.readline()
    if line != '\n':
        i = i+1
        print("%d번째 줄 :"%i + line)
    if i==10:
        break 
f.close()

1번째 줄 :<doc id="5" url="https://ko.wikipedia.org/wiki?curid=5" title="지미 카터">

2번째 줄 :지미 카터

3번째 줄 :제임스 얼 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국의 제39대 대통령(1977년 ~ 1981년) 이다.

4번째 줄 :생애.

5번째 줄 :어린 시절.

6번째 줄 :지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.

7번째 줄 :조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.

8번째 줄 :정계 입문.

9번째 줄 :1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사를 역임했다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.

10번째 줄 :대통령 재임.



# **2. 형태소 분석**

형태소 분석기 Mecab을 사용하여 토큰화를 진행해보겠습니다.



In [ ]:
from konlpy.tag import Mecab 
mecab = Mecab()

In [ ]:
from tqdm import tqdm
from konlpy.tag import Mecab 

In [ ]:
mecab = Mecab()


우선 output_file에는 총 몇 줄이 있는지 확인합니다.



In [ ]:
f = open('output_file.txt', encoding="utf8")
lines = f.read().splitlines()
print(len(lines))

10696334


10,696,334 개의 줄이 존재합니다. 상위 10개만 출력해 봅시다. 


In [ ]:
lines[:10]


['<doc id="5" url="https://ko.wikipedia.org/wiki?curid=5" title="지미 카터">',
 '지미 카터',
 '',
 '제임스 얼 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국의 제39대 대통령(1977년 ~ 1981년) 이다.',
 '생애.',
 '어린 시절.',
 '지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.',
 '조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.',
 '정계 입문.',
 '1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사를 역임했다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.']

두번째 줄을 보면 아무런 단어도 들어있지 않은 ''와 같은 줄도 존재합니다. 해당 문자열은 형태소 분석에서 제외하도록 하고 형태소 분석을 수행해봅시다

In [ ]:
result = []

for line in tqdm(lines):
  # 빈 문자열이 아닌 경우에만 수행
  if line:
    result.append(mecab.morphs(line))

100%|██████████| 10696334/10696334 [15:49<00:00, 11270.32it/s]


빈 문자열은 제외하고 형태소 분석을 진행했습니다. 이제 몇 개의 줄. 즉, 몇 개의 문장이 존재하는지 확인해봅시다.

In [ ]:
len(result)


7228278

# **3. Word2Vec 학습**

형태소 분석을 통해서 토큰화가 진행된 상태이므로 Word2Vec을 학습합니다.



In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(result, size=100, window=5, min_count=5, workers=4, sg=0)

In [ ]:
model_result1 = model.wv.most_similar("대한민국")
print(model_result1)

[('한국', 0.7332375645637512), ('미국', 0.6713722348213196), ('일본', 0.6359390020370483), ('부산', 0.5850777626037598), ('홍콩', 0.5801777243614197), ('태국', 0.5798343420028687), ('오스트레일리아', 0.5686917901039124), ('서울', 0.5551542639732361), ('중화민국', 0.5461504459381104), ('캐나다', 0.5395699739456177)]


In [ ]:
model_result2 = model.wv.most_similar("어벤져스")
print(model_result2)

[('《》,', 0.7922983169555664), ('스파이더맨', 0.7795487642288208), ('엑스맨', 0.7629401683807373), ('어벤저스', 0.7623389959335327), ('인크레더블', 0.7621386051177979), ('트랜스포머', 0.757491946220398), ('퍼니셔', 0.75001460313797), ('솔저', 0.7357286214828491), ('아이언맨', 0.733039379119873), ('킹덤', 0.7329206466674805)]


In [ ]:
model_result3 = model.wv.most_similar("반도체")
print(model_result3)

[('집적회로', 0.7868168354034424), ('실리콘', 0.7677433490753174), ('연료전지', 0.7541879415512085), ('웨이퍼', 0.7514968514442444), ('전자', 0.7290726900100708), ('트랜지스터', 0.7277141809463501), ('PCB', 0.7201352715492249), ('그래핀', 0.7060962319374084), ('변압기', 0.6936673521995544), ('태양광', 0.6915802955627441)]
